In [ ]:
import os
import numpy as np
from scipy.spatial import distance
import matplotlib.pyplot as plt
import sqlite3
import tensorflow as tf
#listing using only 1 GPU
gpus = tf.config.experimental.list_physical_devices("GPU")
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)

2025-03-25 03:33:04.863135: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742873585.566875    4636 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742873585.717727    4636 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1742873587.256755    4636 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1742873587.256785    4636 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1742873587.256786    4636 computation_placer.cc:177] computation placer alr

In [3]:
from tensorflow.keras.applications import VGG16 # type: ignore
from tensorflow.keras.applications.vgg16 import preprocess_input  # type: ignore
from tensorflow.keras.models import Model  # type: ignore
from tensorflow.keras.preprocessing import image  # type: ignore
from tensorflow.keras.preprocessing.image import load_img, img_to_array #type: ignore
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

In [4]:
model = VGG16()
model = Model(inputs=model.inputs, outputs = model.layers[-2].output)
print(model.summary())

I0000 00:00:1742875321.422700    4636 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6687 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080, pci bus id: 0000:01:00.0, compute capability: 6.1


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ cast (Cast)                     │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc1 (Dense)                     │ (None, 4096)           │   102,764,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc2 (Dense)                     │ (None, 4096)           │    16,781,312 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 134,260,544 (512.16 MB)

 Trainable params: 134,260,544 (512.16 MB)

 Non-trainable params: 0 (0.00 B)

None


In [7]:
conn = sqlite3.connect('features.db')
c = conn.cursor()
c.execute('''CREATE TABLE IF NOT EXISTS image_features (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    image_path TEXT,
    label TEXT,
    features TEXT  -- JSON string of the vector
)''')

In [6]:
root_dir = "images"
for class_dir in os.listdir(root_dir):
    class_path = os.path.join(root_dir, class_dir)
    if not os.path.isdir(class_path):
        continue
    for image_file in os.listdir(class_path):
        image_path = os.path.join(class_path, image_file)
        img = load_img(image_path, target_size=(224, 224))
        img_array = img_to_array(img)
        img_array = img_array.reshape((1, *img_array.shape))
        img_array = preprocess_input(img_array)

        features = model.predict(img_array, verbose=0).flatten()
        features_str = ','.join(map(str, features.tolist()))

        # Store in DB
        c.execute("INSERT INTO image_features (image_path, label, features) VALUES (?, ?, ?)",
                  (image_path, class_dir, features_str))

conn.commit()
conn.close()

/mnt/d/Lab/PythonProjects/Multimedia/venv/lib/python3.12/site-packages/keras/src/models/functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor']
Received: inputs=Tensor(shape=(1, 224, 224, 3))
  warnings.warn(msg)
I0000 00:00:1742875335.267563   10367 service.cc:152] XLA service 0x7fb8b8003a60 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1742875335.267608   10367 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce GTX 1080, Compute Capability 6.1
2025-03-25 04:02:15.570014: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1742875336.114970   10367 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-03-25 04:02:20.219748: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:549] Omitted potentially buggy algorithm eng14{} for

In [8]:
batch_size = 32
image_batch = []
path_batch = []
label_batch = []

for image_file in os.listdir(class_path):
    image_path = os.path.join(class_path, image_file)
    img = load_img(image_path, target_size=(224, 224))
    img_array = img_to_array(img)
    image_batch.append(img_array)
    path_batch.append(image_path)
    label_batch.append(class_dir)

    if len(image_batch) == batch_size:
        image_batch_np = preprocess_input(np.array(image_batch))
        features = model.predict(image_batch_np, verbose=0)

        insert_data = []
        for i, feature in enumerate(features):
            feature_str = ','.join(map(str, feature.flatten()))
            insert_data.append((path_batch[i], label_batch[i], feature_str))

        c.executemany("INSERT INTO image_features (image_path, label, features) VALUES (?, ?, ?)", insert_data)
        image_batch = []
        path_batch = []
        label_batch = []

# Don't forget leftover batch
if image_batch:
    image_batch_np = preprocess_input(np.array(image_batch))
    features = model.predict(image_batch_np, verbose=0)

    insert_data = []
    for i, feature in enumerate(features):
        feature_str = ','.join(map(str, feature.flatten()))
        insert_data.append((path_batch[i], label_batch[i], feature_str))

    c.executemany("INSERT INTO image_features (image_path, label, features) VALUES (?, ?, ?)", insert_data)


/mnt/d/Lab/PythonProjects/Multimedia/venv/lib/python3.12/site-packages/keras/src/models/functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor']
Received: inputs=Tensor(shape=(None, 224, 224, 3))
  warnings.warn(msg)
2025-03-25 04:04:23.239813: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:549] Omitted potentially buggy algorithm eng14{} for conv %cudnn-conv-bias-activation.39 = (f16[32,64,224,224]{3,2,1,0}, u8[0]{0}) custom-call(f16[32,3,224,224]{3,2,1,0} %bitcast.514, f16[64,3,3,3]{3,2,1,0} %bitcast.522, f16[64]{0} %bitcast.525), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", metadata={op_type="Conv2D" op_name="functional_1/block1_conv1_1/convolution" source_file="/mnt/d/Lab/PythonProjects/Multimedia/venv/lib/python3.12/site-packages/tensorflow/python/framework/ops.py" source_line=1200}, backend_config={"operation_queue_id":"

In [9]:
from tensorflow.keras import backend as K
import gc

# After a batch or every N images
K.clear_session()
gc.collect()

0